In [4]:
import pyscreenshot as ImageGrab
import keras

Using Theano backend.
Using gpu device 0: GeForce GTX 960 (CNMeM is enabled with initial size: 65.0% of memory, cuDNN not available)


### model

In [8]:
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
from itertools import chain
from keras import backend as K
from keras.layers import Dense, Activation, Convolution2D, Convolution3D, MaxPooling2D, MaxPooling3D, BatchNormalization, Flatten, ZeroPadding2D, Dropout
from keras.models import load_model, Sequential
from keras.optimizers import Adam

INPUT_SHAPE = (3, 455, 590)
# OUTPUT_SHAPE = (1, OUTPUT_ROW, OUTPUT_COL)
OBJECTIVE_FUNCTION = 'binary_crossentropy'
METRICS = ['binary_crossentropy', 'accuracy']

def center_normalize(x):
    """Custom activation for online sample-wise center and std. normalization."""
    return (x - K.mean(x)) / K.std(x)

def VGG_16(weights_path=None):
    model = Sequential()
    model.add(Activation(activation=center_normalize, input_shape=INPUT_SHAPE))
    model.add(ZeroPadding2D((1,1), input_shape=INPUT_SHAPE))
    # model.add(ZeroPadding2D((1,1),input_shape=(2, INPUT_ROW, INPUT_COL)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='softmax'))
    model.compile(optimizer=Adam(lr=1e-5), loss=OBJECTIVE_FUNCTION, metrics=METRICS)

    return model

model = VGG_16()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
activation_2 (Activation)        (None, 3, 455, 590)   0           activation_input_2[0][0]         
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 457, 592)   0           activation_2[0][0]               
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 455, 590)  1792        zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 457, 592)  0           convolution2d_14[0][0]           
___________________________________________________________________________________________

### get screenshot

In [17]:
import numpy as np
import pyscreenshot as ImageGrab
im=ImageGrab.grab(bbox=(120,75,710,530)) # X1,Y1,X2,Y2
# im.show()
im.save("fim.png")
pix = np.array(im)
print(type(pix))
print(pix.shape)
print(pix.mean(), pix.std())

<class 'numpy.ndarray'>
(455, 590, 3)
58.0366536289 67.4806458738


In [6]:
import numpy as np
import pyscreenshot as ImageGrab
im=ImageGrab.grab(bbox=(120,75,710,530)) # X1,Y1,X2,Y2
im.show()
im.save("errou.png")
pix = np.array(im)
print(type(pix))
print(pix.shape)
print(pix.mean(), pix.std())

<class 'numpy.ndarray'>
(455, 590, 3)
58.0361072825 67.4801228299


### joystick

In [13]:
with open('capture.out', 'rb') as f:
    my_list = pickle.load(f)
    print(my_list)

FileNotFoundError: [Errno 2] No such file or directory: 'capture.out'

## this is q-learning or a trial

In [9]:
from time import sleep
from qlearning4k.games.game import Game
from pymouse import PyMouse
from pykeyboard import PyKeyboard
import pyscreenshot as ImageGrab
import numpy as np

k = PyKeyboard()

class Mario(Game):
    SLEEP = 0.1
    ACTIONS = {
        0: 'w',
        1: 's',
        2: 'a',
        3: 'd',
        4: 'f',
        5: 'g',
        6: 'r',
        7: 't',
        8: '\\',
    }
    
    def __init__(self):
        self.reset()
        self.update_state() 
        self.__score = 0

    @property
    def name(self):
        return "Mario"

    @property
    def nb_actions(self):
        return len(self.ACTIONS.values())

    def reset(self):
        k.tap_key(k.function_keys[7])
        
    def update_state(self):
        im = ImageGrab.grab(bbox=(120, 75, 710, 530))
        self.state = np.array(im)
        self.state = self.state.reshape(1, self.state.shape[2], self.state.shape[0], self.state.shape[1])

    def play(self, action):
        print('this is action: ', action)
        k.tap_key(self.ACTIONS[action])
        sleep(self.SLEEP)
        
        self.update_state()
        self.__score += 1

    def get_state(self):
        return self.state

    def get_score(self):
        if self.is_over():
            return 0
        return self.__score
        
    def is_over(self):
        return self.state.mean() <= 1.0

    def is_won(self):
        return self.state.mean() > 1.0 and self.state.mean() < 30.0 and self.state.std() < 50.0

In [12]:
import sys
import os
sys.path.append(os.path.abspath('.'))
from agent import Agent
from time import sleep

sleep(5)

mario = Mario()
agent = Agent(model=model)
agent.train(mario, batch_size=10, nb_epoch=1000, epsilon=.1)
# agent.play(mario)